# Imports

In [1]:
import tensorflow as tf
import larq as lq

from utils import prepare_dataset, generate_data_loaders
from models import generate_quantized_gcn, generate_standard_gcn

# Setup

In [2]:
import warnings
warnings.filterwarnings('ignore')

cora_dataset = prepare_dataset("Cora")
pubmed_dataset = prepare_dataset("PubMed")

cora_tr, cora_va, cora_te = generate_data_loaders(cora_dataset)
pubmed_tr, pubmed_va, pubmed_te = generate_data_loaders(cora_dataset)

Pre-processing node features
Dataset:  cora
Size of train set: 140
Size of val set: 500
Size of test set: 1000
Num classes: 7
Num features: 1433
Pre-processing node features
Dataset:  pubmed
Size of train set: 60
Size of val set: 500
Size of test set: 1000
Num classes: 3
Num features: 500


# Experiment 1: GCN baseline (no quantization)

Following the architecture and hyperparameters settings of Kipf & Welling (2016)

## Cora

In [5]:
input_shapes = (cora_dataset.graphs[0].x.shape[1], cora_dataset.graphs[0].a.shape[1])
model = generate_standard_gcn(channels=64, input_shapes=input_shapes, output_features=cora_dataset.n_labels, dropout_rate=0.5, layers=2)

TypeError: Exception encountered when calling layer "graph_conv_1" (type GraphConv).

in user code:

    File "/Users/tlong/Documents/code/BinaryGNN/models.py", line 17, in call  *
        return tf.sparse.sparse_dense_matmul(self.a, inputs)

    TypeError: Input must be a SparseTensor.


Call arguments received by layer "graph_conv_1" (type GraphConv):
  • inputs=tf.Tensor(shape=(None, 64), dtype=float32)

In [ ]:
type(cora_dataset.graphs[0].a)

## PubMed

In [ ]:
model = generate_standard_gcn(channels=64, input_shapes=data)

# Experiment 2: Quantized GCN Baseline (Same as paper)

# Experiment 3: Dropout - reducing the dropout rate, removing dropout, and putting dropout before vs. after the binarization

# Experiment 4: Allow batchnorm to be trained

# Experiment 5: Batchnorm before every layer

# Experiment 6: Effect of l2 regularization, using binary regularizers

# Experiment 7: Hyperparameter tuning (allow the quantizers to change as part of this)

# Experiment 8: Increase number or width of layers (compare to standard GCN)

# Experiment 9: Use Bop Optimizer

# Experiment 10: Use two-part learning, start with full precision, then do second phase of binarized learning

# Experiment 11: Ternary version